In [74]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
import hook_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [87]:
model: HookedTransformer = HookedTransformer.from_pretrained("gpt2-large")

data = load_dataset("stas/openwebtext-10k", split="train")
data = [item for i, item in enumerate(data["text"]) if len(item) > 2000 and i < 5000]

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


Repo card metadata block was not found. Setting CardData to empty.


In [68]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# colors = px.colors.sequential.Inferno
colors = px.colors.qualitative.Plotly

# Generate x values
x = np.linspace(0, 10, 100)

# Generate y values
y_A = 0.2 + 0.1 * np.sin(x)
y_B = 0.3 + 0.1 * np.sin(2 * x)
y_C = 1 - y_A - y_B

# Create plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y_A, fill='tozeroy', name='A', line=dict(color=colors[5])))
fig.add_trace(go.Scatter(x=x, y=y_A + y_B, fill='tonexty', name='B', line=dict(color=colors[6])))
fig.add_trace(go.Scatter(x=x, y=np.ones_like(x), fill='tonexty', name='C', line=dict(color=colors[2])))

fig.update_layout(
    title="Stacked Chart",
    yaxis=dict(range=[0, 1]),
)
fig.show()

In [72]:
import plotly.express as px
import pandas as pd

colors = px.colors.qualitative.Plotly

df = pd.DataFrame({
    'x': [1, 2, 3, 4],
    'item1': [0.2, 0.3, 0.4, 0.1],
    'item2': [0.4, 0.3, 0.2, 0.5],
    'item3': [0.4, 0.4, 0.4, 0.4]
})

# Create area chart
fig = px.area(df, x='x', y=['item1', 'item2', 'item3'], color_discrete_sequence=[colors[5], colors[6], colors[2]])
fig.update_layout(
    title="Stacked Area Chart",
    yaxis=dict(range=[0, 1]),
)
fig.show()

In [88]:
layer, neuron = 31, 892

hook_name = f'blocks.{layer}.mlp.hook_post'
hooks = [(hook_name, hook_utils.save_activation)]
max_range = np.array([i for i in range(model.cfg.n_ctx)])

prompt_indices = []
token_indices = []
tokens = []
acts = []

for i, prompt in tqdm(enumerate(data)):
    current_tokens = model.to_tokens(prompt)[0]
    with model.hooks(hooks):
        model(current_tokens)

    current_acts = model.hook_dict[hook_name].ctx['activation'][0, :, neuron]

    prompt_indices.append(np.array([i]*current_tokens.shape[0]))
    token_indices.append(max_range[:current_tokens.shape[0]])
    tokens.append(current_tokens.cpu().numpy())
    acts.append(current_acts.cpu().numpy())

0it [00:00, ?it/s]

In [89]:
prompt_indices = np.concatenate(prompt_indices)
token_indices = np.concatenate(token_indices)
tokens = np.concatenate(tokens)
acts = np.concatenate(acts)

df = pd.DataFrame({
    'prompt': prompt_indices,
    'token': token_indices,
    'token_str': tokens,
    'activation': acts
})

In [ ]:
# df?
# np.*load*?

In [90]:
df_shuffled = df.sample(frac=1, random_state=0).reset_index(drop=True)
df_shuffled['bucket'] = pd.cut(df_shuffled['activation'], bins=20, labels=False)
print(df_shuffled.head())

sample_df = df_shuffled.groupby('bucket').apply(lambda x: x.sample(n=min(len(x), 100), random_state=0)).reset_index(drop=True)
print(sample_df.groupby('bucket').apply(lambda x : len(x)))

   prompt  token  token_str  activation  bucket
0    3040    337       6291   -0.160312       0
1    2973    255      18685   -0.052533       0
2    2984    361        286   -0.118980       0
3    1980    240        447   -0.000000       0
4    3040    782      27008   -0.106191       0
bucket
0     100
1     100
2     100
3     100
4     100
5     100
6     100
7      97
8      63
9      43
10     31
11     25
12     17
13     15
14     26
15     30
16     17
17      6
18      1
19      1
dtype: int64


In [91]:
with open('data/gpt2_large_spectrum.pkl', 'wb') as f:
    pickle.dump(sample_df, f)


In [93]:
sample_df = df_shuffled.groupby('bucket').apply(lambda x: x.sample(n=min(len(x), 20), random_state=0)).reset_index(drop=True)
sample_df

,prompt,token,token_str,activation,bucket
0,2843,549,749,-0.006965,0
1,662,586,7998,-0.008924,0
2,1571,201,3340,-0.009869,0
3,47,45,1367,-0.027109,0
4,752,163,3199,0.006339,0
...,...,...,...,...,...
332,1919,688,2641,12.707108,17
333,3259,556,11,12.589984,17
334,962,483,423,13.138277,17
335,201,661,423,13.994400,18


In [98]:
for i, row in sample_df.iterrows():
    print(i, type(row)) 
    prompt = data[row['prompt_index']]
    print(prompt)
    break

0 <class 'pandas.core.series.Series'>


KeyError: 'prompt_index'